In [2]:
import sys
[sys.path.append(i) for i in ['.', '..']]
import torch
import numpy as np
import sigpy as sp
from prox import Prox
from linear.linearmaps import LinearMap

In [1]:
from skimage.data import shepp_logan_phantom

In [3]:
A = LinearMap([5,5], [5,5])

In [4]:
B = LinearMap([5,5], [4,5])

In [8]:
C = A*5

In [9]:
C.size_in

[5, 5]

In [10]:
C.size_out

[5, 5]

In [6]:
a = shepp_logan_phantom(200)

TypeError: shepp_logan_phantom() takes 0 positional arguments but 1 was given

In [5]:
a.shape

(400, 400)